# 1. Threads y PyQt

```python
import sys
from time import sleep
from PyQt6.QtCore import pyqtSignal, QThread
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar):
        super().__init__()
        self.senal_actualizar = senal_actualizar

    def run(self):
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))

        sleep(0.5)
        self.senal_actualizar.emit("Status: Qthread terminado")


class MiVentana(QWidget):
    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_labels)

        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel("Status: esperando Qthread", self)
        self.boton = QPushButton("Ejecutar QThread", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        self.label.setGeometry(10, 10, 230, 30)
        self.boton.setGeometry(10, 50, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qthreads")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        # Aquí debemos ocupar isRunning en lugar de is_alive
        if self.thread is None or not self.thread.isRunning():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento)

```

1. Ejemplo de un thread usando en el contexto de PyQt
- Un thread que se encarga de emitir una señal cada 0.5 segundos

- En la ventana principal, asociar esa señal a un metodo que reciba la info procesada por el thread

- Crear un widget que se conecte a un metodo que inicie el thread
- El thread inicia y va emitiendo la señal
- El metodo asociado a la señal se va actualizando

## QThread

- Lo mismo que un thread, pero creado para funcionar dentro de PyQt
- Se define igual. Se puede hacaer *overriding* de `run()`
- `is_alive` de `threading` es equivalente al metodo `isRunning` 

## QMutex

- lo mismo que un lock. tiene como metodos `lock` y `unlock`

# 2. QTimer

- ejecuta una subrutina cada cierto tiempo


- `setInterval(ms)`: periodo ejecuciones
- `timeout.connect(func/mehtod)`: conexion a la subrutina que se quiere ejecutar
- `start()`: inicia el timer
- `stop()`: detiene la siguente ejecucion del el timer, NO interrumpe el timer, 

## `singleShot`

- para situaciones donde queremos que el timer ejecute du funcion **una unica vez** despues de transcurrido un tiempo determinado
- `singleShot(bool)`: indica al timer que este debe ejecutarse 1 vez y luego detenerse
- el tiempo de espera se indica con `setInterval(ms)`

## Extra

- Correcta modularizacion permite reutilizar un front end
- cuando se crea un label que va a cambiar, crear en el backend una contrapartr que se preocupe de gestionarlo, puede ser un `QThread`

# 3. Miscelaneos

## Usar señales siempre es lo ideal

## *isAutoRepeat* en *KeyPressEvent*

- `isAutoRepeat` entrega un booleano si es que la tecla fue presionada por 1ra vez (True) o si esta siendo mantenida presionada (False)

In [ ]:
def keyPressEvent(self, event):
        if event.key() == Qt.Key.Key_W:
            self.contador_w += 1
            self.label_w_contador.setText(f"Presionada {self.contador_w} veces")

        # si la tecla es la A si no fue mantenida (su registro no viene de haberla
        # mantenido presionada)
        if event.key() == Qt.Key.Key_A and not event.isAutoRepeat():
            self.contador_a += 1
            self.label_a_contador.setText(f"Presionada {self.contador_a} veces")

## Sonidos en PyQT

- Se debe importar

```python
from PyQt6.QtMultimedia import QMediaPlayer, QAudioOutput, QSoundEffect
from PyQt6.QtCore import QUrl


```

### `QMediaPlayer`

- para reproducir archivos .mp3
1. Instanciar el objeto: `self.media_player_mp3 = QMediaPlayer(self)`
2. Entregarle un objeto tipo QAudioOutput: `self.media_player_mp3.setAudioOutput(QAudioOutput(self))`. Esto es para que el reproductor entienda que va a desplegar un sonido.
3. Definir un objeto tipo `QURL` con el paths a nuestro sonido: `file_url = QUrl.fromLocalFile(join("sounds", "waku-waku.mp3"))`.
4. Entregarle la URL a nuestro reproductor: `self.media_player_mp3.setSource(file_url)`
5. Finalmente usar `play()` para reproducir el sonido.

### `QSoundEffect`

- para reproducir archivos .wav

1. Instanciar el objeto: `self.media_player_wav = QSoundEffect(self)`
2. Definir un objeto tipo `QURL` con el paths a nuestro sonido: `file_url = QUrl.fromLocalFile(join("sounds", "see-you-again.wav"))`.
3. Entregarle la URL a nuestro reproductor: `self.media_player_wav.setSource(file_url)`
4. Finalmente usar `play()` para reproducir el sonido. También podemos usar `stop()` para detenerlo.

# Apuntes clase

![Alt text](image.png)

# 4. Main Window

- Barra de menu: grupo de comandos organizados

- barra de herramientas: acceso rapido a los comanmdos mas utilizados

- central widget: cuerpo de la ventana, para agregarle widgets se usa `setCentralWidget(widget)`

- Barra de estado: muestra info del estado de la aplicacion. `QApplication.statusBar()`

### Acciones

- Creamos acciones (comandos) con `QAction`
- indicamos que hacen con `setStatusTip`, util para la barra de estado

- usamos `accion.triggered.conect()` para conectar la accion a un metodo que queramos que se ejecute

```python
"""Configuramos las acciones."""
ver_status = QAction(QIcon(None), "&CambiaStatus", self)
ver_status.setStatusTip("Este es un ítem de prueba")
ver_status.triggered.connect(self.cambiar_status_bar)

limpiar_status = QAction(QIcon(None), "&LimpiaStatus", self)
limpiar_status.setStatusTip("Esta acción limpia la barra de estado")
limpiar_status.triggered.connect(self.limpiar_status_bar)

buscar = QAction(QIcon(os.path.join("img", "search_icon.png")), "&Search", self)
buscar.setStatusTip("Un ícono de búsqueda")

salir = QAction(QIcon(None), "&Exit", self)
salir.setShortcut("Ctrl+Q")
salir.setStatusTip("Salir de la aplicación")
salir.triggered.connect(QApplication.quit)

```

### Barra de Menu
- creamos una barra de menu con `menuBar()`
- le añadimos submenus con `menuBar.addMenu(name)`

```python

menubar = self.menuBar()
archivo_menu = menubar.addMenu("Archivo")  # primemenú
archivo_menu.addAction(ver_status)
archivo_menu.addAction(salir)
```

### Barra de herramientas
- creamos una barra de herramientas con `addToolBar(name)`


```python
toolbar = self.addToolBar("Toolbar")
toolbar.addAction(buscar)
toolbar.addAction(salir)
```

- para ambos, añadimos acciones con `addAction`